In [ ]:
import h5py

def inspect_h5_file(file_path):
    with h5py.File(file_path, 'r') as file:
        print("Keys in the HDF5 file:")
        print(list(file.keys()))

inspect_h5_file('/content/drive/MyDrive/Train.h5')
inspect_h5_file('/content/drive/MyDrive/Test.h5')

Keys in the HDF5 file:
['images', 'labels']
Keys in the HDF5 file:
['images', 'labels']


1. Load and preprocess the data.

In [ ]:
import numpy as np
def load_data(train_path, test_path):
    with h5py.File(train_path, 'r') as train_file:
        X_train = np.array(train_file['images'][:])
        y_train = np.array(train_file['labels'][:])
    with h5py.File(test_path, 'r') as test_file:
        X_test = np.array(test_file['images'][:])
        y_test = np.array(test_file['labels'][:])
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_data('/content/drive/MyDrive/Train.h5', '/content/drive/MyDrive/Test.h5')

In [ ]:
def preprocess_data(X):
    X_flatten = X.reshape(X.shape[0], -1).T
    X_norm = X_flatten / 255.
    return X_norm

X_train = preprocess_data(X_train)
X_test = preprocess_data(X_test)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (49152, 2626)
Shape of y_train: (2626,)


In [ ]:
def one_hot_encoding(y, num_classes):
    m = y.shape[0]
    one_hot = np.zeros((num_classes, m))
    one_hot[y, np.arange(m)] = 1
    return one_hot

In [ ]:
# Assuming the labels range from 0 to num_classes-1
num_classes = np.max(y_train) + 1
y_train_one_hot = one_hot_encoding(y_train, num_classes)  # Shape should be (5, 2626)
y_test_one_hot = one_hot_encoding(y_test, num_classes)    # Shape should be (5, number_of_test_samples)

print("Shape of y_train_one_hot:", y_train_one_hot.shape)

Shape of y_train_one_hot: (5, 2626)


2. Implement the softmax activation function.

In [ ]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

3. Define the cost function for multi-class classification.

In [ ]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(AL + 1e-8)) / m
    return cost

4. Implement the gradient descent algorithm to update the model parameters.

In [ ]:
def gradient_descent(X, Y, learning_rate, num_iterations):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    m = X.shape[1]

    W = np.random.randn(n_y, n_x) * 0.01
    b = np.zeros((n_y, 1))

    for i in range(num_iterations):
        Z = np.dot(W, X) + b
        A = softmax(Z)

        cost = compute_cost(A, Y)

        dZ = A - Y
        dW = np.dot(dZ, X.T) / m
        db = np.sum(dZ, axis=1, keepdims=True) / m

        W -= learning_rate * dW
        b -= learning_rate * db

        if i % 100 == 0:
            print(f"Cost after iteration {i}: {cost}")

    return W, b

5. Test the model on the test set and evaluate its performance.

In [ ]:
def predict(X, W, b):
    Z = np.dot(W, X) + b
    AL = softmax(Z)
    return np.argmax(AL, axis=0)

In [ ]:
# Train the model on lr = 0.01 and iterations = 1000
W, b = gradient_descent(X_train, y_train_one_hot, learning_rate=0.01, num_iterations=1000)

# Test the model
predictions = predict(X_test, W, b)
accuracy = np.mean(predictions == np.argmax(y_test_one_hot, axis=0))
print(f"Accuracy on test set: {accuracy * 100}%")

Cost after iteration 0: 2.2909326209920553
Cost after iteration 100: 8.710073597059846
Cost after iteration 200: 9.962969627141593
Cost after iteration 300: 7.628248610735271
Cost after iteration 400: 5.206622414683755
Cost after iteration 500: 3.23503395593863
Cost after iteration 600: 5.73247481443108
Cost after iteration 700: 4.412486032726907
Cost after iteration 800: 7.544214400067103
Cost after iteration 900: 7.934580078989827
Accuracy on test set: 44.166666666666664%


In [ ]:
# Train the model on lr = 0.001 and iterations = 2000
W, b = gradient_descent(X_train, y_train_one_hot, learning_rate=0.001, num_iterations=2000)

# Test the model
predictions = predict(X_test, W, b)
accuracy = np.mean(predictions == np.argmax(y_test_one_hot, axis=0))
print(f"Accuracy on test set: {accuracy * 100}%")

Cost after iteration 0: 1.7753530391002523
Cost after iteration 100: 1.2917167718358047
Cost after iteration 200: 1.212072444970203
Cost after iteration 300: 1.1648535879031712
Cost after iteration 400: 1.1291825994721794
Cost after iteration 500: 1.0994674881862083
Cost after iteration 600: 1.0735010905263709
Cost after iteration 700: 1.0501835489603142
Cost after iteration 800: 1.0288760919097169
Cost after iteration 900: 1.0091674212180248
Cost after iteration 1000: 0.9907731624774994
Cost after iteration 1100: 0.9734864590034663
Cost after iteration 1200: 0.9571510736762607
Cost after iteration 1300: 0.9416455257845279
Cost after iteration 1400: 0.9268731335555236
Cost after iteration 1500: 0.9127554515794882
Cost after iteration 1600: 0.89922777664378
Cost after iteration 1700: 0.8862359744208581
Cost after iteration 1800: 0.8737341821951362
Cost after iteration 1900: 0.8616831107216487
Accuracy on test set: 49.166666666666664%
